# Get Ford dealer locations

#### Load Python tools and Jupyter config

In [1]:
import us
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
place = "ford"
place_formal = "Ford"
color = "#0068d0"
latitude = "39.106667"
longitude = "-94.676392"

## Scrape

#### Headers for the request

In [73]:
headers = {
    "authority": "www.ford.com",
    "accept": "application/json, text/javascript, */*; q=0.01",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    "application-id": "07152898-698b-456e-be56-d3d83011d0a6",
    "referer": "https://www.ford.com/dealerships/",
    "sec-ch-ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "x-dtpc": "8$163699064_118h35vQAHCMNCTDHWFLMGAMWDHNBWEUFUNHMPA-0e0",
    "x-dtreferer": "https://www.ford.com/dealerships/",
    "x-requested-with": "XMLHttpRequest",
}

params = {
    "make": "Ford",
    "radius": "500",
    "minDealers": "10",
    "maxDealers": "100",
    "postalCode": "64030",
}

response = requests.get(
    "https://www.ford.com/cxservices/dealer/Dealers.json",
    params=params,
    cookies=cookies,
    headers=headers,
)

In [74]:
src = pd.DataFrame(response.json()["Response"]["Dealer"])[
    [
        "Name",
        "Address",
        "Latitude",
        "dlrcalltrk_lad",
        "Longitude",
        "URL",
        "Phone",
    ]
]
src.columns = src.columns.str.lower()

In [75]:
src[["Street2", "Street1", "State", "PostalCode", "Country", "City", "Street3"]] = (
    pd.json_normalize(src["address"])
)

In [76]:
df = (
    src.drop(["address", "Street2", "Country", "Street3"], axis=1)
    .rename(
        columns={
            "dlrcalltrk_lad": "phone",
            "State": "state",
            "Street1": "street",
            "PostalCode": "zip",
            "City": "city",
        }
    )
    .copy()
)

In [77]:
df.head()

,name,latitude,phone,longitude,url,phone,street,state,zip,city
0,Rob Sight Ford,38.87598,(816) 477-7558,-94.602955,https://www.robsightfordkc.com,(816) 941-1200,13901 Washington Street,MO,64145,Kansas City
1,Bob Sight Ford Inc,38.92455,(816) 477-5711,-94.402551,https://www.bobsightford.com,(816) 524-6550,610 NW Blue Parkway,MO,64063,Lees Summit
2,Greenway Ford,38.991501,(816) 607-9029,-94.46931,https://www.greenwayfordraytown.net,(816) 353-1495,9505 E 350 Hwy,MO,64133,Raytown
3,"Bob Allen Ford, Inc.",38.961028,(816) 477-7574,-94.667414,https://www.boballenonline.com,(913) 381-3000,9239 Metcalf Ave,KS,66212,Overland Park
4,Shawnee Mission Ford,39.014401,(816) 477-5041,-94.71972,https://www.shawneemissionford.com,(913) 631-0000,11501 W. Shawnee Mission Pkwy,KS,66203,Shawnee


In [78]:
len(df)

100